This script is designed to enrich an Excel file with data from an ArcGIS Online feature service. The purpose of the script is to perform a join operation between the data in the Excel file and the data in the feature service, and then save the result to a new Excel file.

- parametrerize
- make atbx
- test

In [94]:
# inspect the layers associated with an item id

import pandas as pd
from arcgis.features import FeatureLayer
from arcgis.gis import GIS

def get_item_dfs(item_id):
    gis = GIS("home")

    # Get the item by its ID
    item = gis.content.get(item_id)

    # If the item is found and it's a feature service, convert each layer to a DataFrame
    if item and item.type == "Feature Service":
        dfs = []
        for i in range(len(item.layers)):
            layer = FeatureLayer(item.url + f"/{i}")
            sdf = layer.query().sdf
            df = sdf.drop(columns=['SHAPE'])
            dfs.append((layer.properties.name, layer.properties.description, df))
        return dfs

    # If the item is not found or it's not a feature service, return None
    return None

# Call the function with an example
dfs = get_item_dfs('a8147954b9ac4e9cb717f344b1d87b70')

# If DataFrames are returned, print the fields and their properties of each one
if dfs is not None:
    for i, (name, description, df) in enumerate(dfs):
        print(f"DataFrame for layer {i} ({name}):")
        print(f"Description: {description}")
        print("Fields and their properties:")
        print(df.dtypes)



DataFrame for layer 0 (cGIS Distribution Circuits GG Concat):
Description: 
Fields and their properties:
OBJECTID                Int64
CITY                   string
COMPUTED_LENGTH       Float64
E_SWITCHING_CENTER     string
SYS_STATUS             string
                       ...   
ID_FEEDER               Int32
MAP3D_CIRCUIT_NAME     string
CIRCUIT_OH_UG          string
Shape__Length         Float64
GlobalID               string
Length: 82, dtype: object


In [95]:
# get the url from an item id and layer index

from arcgis.gis import GIS

def get_layer_url(item_id,i):
    gis = GIS("home")

    # Get the item by its ID
    item = gis.content.get(item_id)

    # If the item is found, return its URL with a 0 to refer to the first layer
    if item and item.type == "Feature Service":
        return item.url + f"/{i}"

    # If the item is not found, return None
    return None

item_url = get_layer_url('a8147954b9ac4e9cb717f344b1d87b70',0) # cGIS Transmission Lines (GG CONCAT)
print(item_url)


https://services5.arcgis.com/z6hI6KRjKHvhNO0r/arcgis/rest/services/cGIS_Distribution_Circuits_GG_Concat/FeatureServer/0


```python
def enrich_excel(excel_file, in_join_field, spatial_layer_url, join_table_field, fields_to_add=None, output_file=None):
```

This function enriches an Excel file with additional data from a spatial layer and saves the result to a new Excel file. The function performs a join operation between the data in the Excel file and the spatial layer based on specified fields.

**Parameters:**

- `excel_file` (str): The path to the Excel file that you want to enrich. This file should be in .xlsx format.

- `in_join_field` (str): The name of the field in the Excel file that will be used for the join operation. This field should contain unique identifiers that match those in the `join_table_field` of the spatial layer.

- `spatial_layer_url` (str): The URL of the spatial layer that contains the additional data you want to add to the Excel file. This should be a valid URL of a feature service.

- `join_table_field` (str): The name of the field in the spatial layer that will be used for the join operation. This field should contain unique identifiers that match those in the `in_join_field` of the Excel file.

- `fields_to_add` (list, optional): A list of field names from the spatial layer that you want to add to the Excel file. If this parameter is not provided, all fields from the spatial layer will be added.

- `output_file` (str, optional): The name of the output Excel file. If this parameter is not provided, a default name will be created based on your input Excel file and the fields you're adding.

**Returns:**

- str: The path to the enriched Excel file.

**Example usage:**

```python
enriched_excel = enrich_excel('example.xlsx', 'FLOC', get_item_url('a8147954b9ac4e9cb717f344b1d87b70'), 'SCE_FLOC', ['CIRCUIT_NAME', 'CIRCUIT_NO'])
```

In this example, 'example.xlsx' is enriched with data from a spatial layer at a given URL. The join operation is performed based on 'FLOC' field in 'example.xlsx' and 'SCE_FLOC' field in the spatial layer. Only 'CIRCUIT_NAME' and 'CIRCUIT_NO' fields from the spatial layer are added to 'example.xlsx'. The result is saved as 'example_enriched_with_CIRCUIT_NAME_CIRCUIT_NO.xlsx'.

In [91]:
import pandas as pd
from arcgis.features import FeatureLayer

def enrich_excel(excel_file, in_join_field, spatial_layer_url, join_table_field, fields_to_add=None, output_file=None):
    # Read the Excel file into a DataFrame
    df = pd.read_excel(excel_file)

    # Query the feature service to get a SpatialDataFrame
    sdf = FeatureLayer(spatial_layer_url).query().sdf

    # Perform the join operation
    result = pd.merge(df, sdf, left_on=in_join_field, right_on=join_table_field)

    # If specific fields are provided, select only those fields from the result
    if fields_to_add is not None:
        result = result[df.columns.tolist() + fields_to_add]

    # Drop duplicate records
    result = result.drop_duplicates()

    # If no output file name is provided, create a default one
    if output_file is None:
        base_name = excel_file.rsplit('.', 1)[0]
        fields_str = '_'.join(fields_to_add) if fields_to_add else 'all_fields'
        output_file = f"{base_name}_enriched_with_{fields_str}.xlsx"

    # Save the DataFrame to an Excel file
    result.to_excel(output_file, index=False)

    # Return the enriched Excel file
    return output_file

# Call the function with an example
enriched_excel = enrich_excel('example.xlsx', 'FLOC', get_item_url('a8147954b9ac4e9cb717f344b1d87b70'), 'SCE_FLOC', ['CIRCUIT_NAME', 'CIRCUIT_NO'])